In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [89]:
data = pd.read_csv("Churn_Modelling.csv")

In [90]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [92]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [93]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [94]:
#Initial 3 columns are not addig value, hence lets drop them
data = data.iloc[0: , 3:]

In [95]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [96]:
# as we can see, we have 2 categorical columns. Lets address them
# Lets understand the unique values in "Gender" columns
data.Gender.value_counts()
data.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [97]:
# Lets use LabelEncoder to convert these columns
label_encoder_gender = LabelEncoder()
data["Gender"] =label_encoder_gender.fit_transform(data["Gender"])


In [98]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [99]:
geo_encoder.toarray()    #This value should be passed while converting to DataFrame

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [100]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [101]:
# Lets convert it to a DataFrame
geo_encoded_df  = pd.DataFrame(geo_encoder.toarray(), columns= onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [102]:
data = pd.concat([data.drop (["Geography"], axis = 1),geo_encoded_df], axis = 1 )

In [103]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [104]:
# Save the encoders & scalers for later use
with open("lable_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)
with open("onehot_encoder_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)

In [105]:
# Devide the data set into dependent & independent feature
X = data.drop(["Exited"], axis=1)
y= data["Exited"]

In [106]:
# Lets split the data into Training/Testing
X_train,X_test, y_train, y_test = train_test_split(X,y,train_size= 0.8, random_state=54)

# Also, we have to scale down the values.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [107]:
# Lets also save this in Pickel file
with open ("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [108]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [109]:
# Lets Build our ANN model
model = Sequential([

    Dense(64,activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation= "relu"),
    Dense(1, activation="sigmoid")
]
)

In [110]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
# loss = tensorflow.keras.losses.BinaryCrossEntropy()

In [112]:
# lets compile the model
model.compile(optimizer=opt, loss = "binary_crossentropy" , metrics=['accuracy'])

In [113]:
# Set up the TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [114]:
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [115]:
# Lets build the model
history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback] )    

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.3972 - accuracy: 0.8356 - val_loss: 0.3436 - val_accuracy: 0.8675
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3634 - accuracy: 0.8505 - val_loss: 0.3307 - val_accuracy: 0.8655
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3521 - accuracy: 0.8549 - val_loss: 0.3295 - val_accuracy: 0.8700
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3444 - accuracy: 0.8577 - val_loss: 0.3378 - val_accuracy: 0.8665
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3431 - accuracy: 0.8577 - val_loss: 0.3377 - val_accuracy: 0.8565
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3400 - accuracy: 0.8596 - val_loss: 0.3387 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3363 - accuracy: 0.8620 - val_loss: 0.3457 - val_accuracy: 0.8650

In [116]:
model.save('model.h5')

c:\Users\Ritesh\Desktop\ML\Projects\Krish Naik - GenAI\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [117]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [118]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 22272), started 20:46:17 ago. (Use '!kill 22272' to kill it.)